In [1]:
import yt
from matplotlib import rc_context
from matplotlib.animation import FuncAnimation

In [8]:
unit_base={"length_unit": (1.0,"pc"), "time_unit": (1.0,"s*pc/km"), "mass_unit": (2.38858753789e-24,"g/cm**3*pc**3")}
# ts = yt.load("kh_custom.out1.*.athdf", units_override=unit_base)

In [6]:
# Create a sample SlicePlot (this indirectly creates ImagePlot objects)
ds = yt.load_sample("IsolatedGalaxy")
plot = yt.SlicePlot(ds, "z", ("gas", "density"))

# List all attributes and methods for the ImagePlot object
print(dir(plot.plots[("gas", "density")]))

/Users/rhsu/micromamba/envs/shearstream/lib/python3.10/site-packages/yt/sample_data/api.py:149: UserWarning: Storage directory from yt config doesn't exist (currently set to '/does/not/exist'). Current working directory will be used instead.
  warn(
yt : [INFO     ] 2024-09-09 16:52:15,163 Sample dataset found in '/Users/rhsu/shearstream/IsolatedGalaxy/galaxy0030/galaxy0030'
yt : [INFO     ] 2024-09-09 16:52:15,214 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2024-09-09 16:52:15,214 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2024-09-09 16:52:15,214 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-09-09 16:52:15,215 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-09-09 16:52:15,215 Parameters: cosmological_simulation   = 0
Parsing Hierarchy : 100%|██████████| 173/173 [00:00<00:00, 31398.29it/s]
yt : [INFO     ] 2024-09-09 16:52:15,225 Gathering a field list (this may take a moment.)


['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_aspect', '_ax_text_size', '_cb_size', '_create_axes', '_default_font_size', '_draw_axes', '_draw_frame', '_figure_size', '_get_best_layout', '_get_canvas_classes', '_get_labels', '_init_image', '_plot_valid', '_projection', '_repr_png_', '_reset_layout', '_set_axes', '_set_font_properties', '_setup_layout_constraints', '_toggle_axes', '_toggle_colorbar', '_top_buff_size', '_transform', '_unit_aspect', '_validate_axes_extent', 'axes', 'canvas', 'cax', 'cb', 'colorbar_handler', 'figure', 'hide_axes', 'hide_colorbar', 'image', 'interactivity', 'norm_handler', 'save', 'show', 'show_axes', 'show_colorb

In [15]:
ds = yt.load("kh-shear.out2.00050.athdf", units_override=unit_base)

yt : [WARNING  ] 2024-09-09 17:13:09,771 Overriding code units: Use this option only if you know that the dataset doesn't define the units correctly or at all.
yt : [INFO     ] 2024-09-09 17:13:09,771 Overriding length_unit: 1.0 pc.
yt : [INFO     ] 2024-09-09 17:13:09,771 Overriding time_unit: 1.0 pc*s/km.
yt : [INFO     ] 2024-09-09 17:13:09,772 Overriding mass_unit: 2.38858753789e-24 g*pc**3/cm**3.
yt : [WARNING  ] 2024-09-09 17:13:09,773 Assuming 1.0 = 1.0 K
yt : [INFO     ] 2024-09-09 17:13:09,800 Parameters: current_time              = 5.0
yt : [INFO     ] 2024-09-09 17:13:09,800 Parameters: domain_dimensions         = [256 256   1]
yt : [INFO     ] 2024-09-09 17:13:09,801 Parameters: domain_left_edge          = [-0.5 -0.5 -0.5]
yt : [INFO     ] 2024-09-09 17:13:09,801 Parameters: domain_right_edge         = [0.5 0.5 0.5]
yt : [INFO     ] 2024-09-09 17:13:09,801 Parameters: cosmological_simulation   = 0


In [19]:
ds.r["density"]

unyt_array([1.01144922, 1.01144886, 1.01144814, ..., 1.0165801 , 1.01658726,
       1.01659131], 'code_mass/code_length**3')